# Downward inference
In this jupyter notebook we will show simple example downward inference in [Logical Neural Networks](https://github.com/IBM/LNN) (LNN).


## The basics
Before we start with downward inference example in LNN we will describe the idea behind it. In simple case of one [*logical formula*](https://en.wikipedia.org/wiki/First-order_logic#Formulas) with [*logical equality*](https://en.wikipedia.org/wiki/Logical_equality) (also called Bidirectional implication, since it can be expressed as two logical implications and logical and) the downward inference is evaluation of the *operand* given the truth of the *operator* and rest of the operands.

The logical equality is evaluated based on its *truth table* which can be seen below.
![equality_tt](images/equality_tt.png)

So if we want to do downward inference on $P(x) \Leftrightarrow Q(x)$ we have to provide truth value of the logical equality operator and one of the operands. Lets say that whole formula is evaluated to $False$ and truth value of predicate $P(x)$ is $True$. Situation is shown on image below.

![downward_inference](images/downward_inference.png)

Now we can *infer* the truth value of predicate $Q(x)$ based on value of $P(x)$ and value of $P(x) \Leftrightarrow Q(x)$. Since our operator is logical equality we can easily use truth table above and state that value of $Q(x)$ must be $False$.

We can apply downward inference to multiple values of our predicates:

![downward_inference](images/predicate_down.png)


## Grecian domain

Now we are ready to move to our specific example and finally create a LNN model. We will continue with our grecian domain. We will extend it by new [axiom](https://en.wikipedia.org/wiki/Axiom). This axiom will be based on [ontology for word "grecian"](http://wordnetweb.princeton.edu/perl/webwn?o2=&o0=1&o8=1&o1=1&o7=&o5=&o9=&o6=&o3=&o4=&s=grecian&i=0&h=00#c). Let us modify description of our domain in natural language:

* All grecian is a native or resident of Greece.<br>
* Grecians are mortal.<br>
* Zeus is resident of Greece (is not native of Greece), is not mortal.<br>
* Socrates is native of Greece (is not resident of Greece), is mortal.<br>
* Confucius is neither native or resident of Greece, is mortal.<br>
* Great Pyramid of Giza is neither native or resident of Greece, is not mortal.

Now we can formulate `All grecian is a native or resident of Greece` in context of First-order logic:

$(\forall x) (native\_of\_greece(x) \lor resident\_of\_greece(x)) \Leftrightarrow grecian(x)$

Before we move on, we have to explain multiple things.

**Axiom:**
When we state that some formula is axiom we say that is taken to be true statemen in our domain. We can use this knowledge as a ground for infering future informations.

**Universal quantificator:**
We used [*universal quantificator*](https://en.wikipedia.org/wiki/Universal_quantification) $\forall x$ to state that all objects from our domain have to satisfy our statement that `All grecian is a native or resident of Greece`.

**Two new predicates:**
We created two new predicates $native\_of\_greece(x)$ and $resident\_of\_greece(x)$. Since current one were not enought to express our domain from natural language description.

**Logical disjunction:**
We used new logical connective called [*logical didisjunction*](https://en.wikipedia.org/wiki/Logical_disjunction) which represents "or" in natural language.

So our new situation can be described by following table:

![grecian_downward_tt](images/grecian_downward_tt.png)

We would like to infer values for $grecian(x)$ later.

## Definition of LNN model

Now we can continue with our LNN model definition.

In [24]:
from lnn import Model

model = Model()

Define our predicates and add them to the model.

In [25]:
from lnn import Predicate

grecian = Predicate(name="grecian")
mortal = Predicate(name="mortal")
native_of_greece = Predicate(name="native_of_greece")
resident_of_greece = Predicate(name="resident_of_greece")

model.add_formulae(grecian, mortal, native_of_greece, resident_of_greece)

Create free variable and our formulae. Note that we set our axiom as `AXIOM`.

In [26]:
from lnn import Variable, Implies, World, ForAll, Or, Bidirectional

x = Variable("x")

all_grecians_are_native_or_resident = ForAll(
    x,
    Bidirectional(
        Or(native_of_greece(x), resident_of_greece(x), name="native_or_resident"),
        grecian(x),
        name="grecians_are_native_or_resident",
    ),
    name="all_grecians_are_native_or_resident",
    world=World.AXIOM,
)

grecians_are_mortals = Implies(grecian(x), mortal(x), name="grecians_are_mortals")

model.add_formulae(grecians_are_mortals, all_grecians_are_native_or_resident)


Add facts provided by descripton.

In [27]:
from lnn import Fact

model.add_facts(
    {
        "native_of_greece": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.FALSE,
            "Zeus": Fact.FALSE,
            "Socrates": Fact.TRUE,
        },
        "resident_of_greece": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.FALSE,
            "Zeus": Fact.TRUE,
            "Socrates": Fact.FALSE,
        },
        "mortal": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.TRUE,
            "Zeus": Fact.FALSE,
            "Socrates": Fact.TRUE,
        },
    }
)

### Downward inference inside LNN model

Before we start downward inference we have to evaluate maximum number of formulae given their operands (facts from predicates). So we will run upward inference first.

In [28]:
from lnn import Direction

model.infer(direction=Direction.UPWARD)

(1, tensor(10., grad_fn=<AddBackward0>))

Now we can verify that upward inference is not enough to infer facts of our $grecian(x)$ predicate.

In [29]:
model["grecian"].print()

OPEN   Predicate: grecian(x0) 
'Great Pyramid of Giza'                                  UNKNOWN (0.0, 1.0)
'Zeus'                                                   UNKNOWN (0.0, 1.0)
'Socrates'                                               UNKNOWN (0.0, 1.0)
'Confucius'                                              UNKNOWN (0.0, 1.0)



To infer it, we have to start downward inference which will use truth values of formulae which were calculated in upward inference to infer truth values of our facts.

In [30]:
model.infer(direction=Direction.DOWNWARD)

(1, tensor(12., grad_fn=<AddBackward0>))

And as we can see, model correctly infered truth value for $grecian(x)$ predicate.

In [31]:
model["grecian"].print()

OPEN   Predicate: grecian(x0) 
'Great Pyramid of Giza'                                    FALSE (0.0, 0.0)
'Zeus'                                                      TRUE (1.0, 1.0)
'Socrates'                                                  TRUE (1.0, 1.0)
'Confucius'                                                FALSE (0.0, 0.0)



How was that possible? Remember the example from the beginning. We already know the truth value for formula with logical disjuntion (or):

In [32]:
model["native_or_resident"].print()

OPEN   Or: native_or_resident(x) 
'Great Pyramid of Giza'                                    FALSE (0.0, 0.0)
'Zeus'                                                      TRUE (1.0, 1.0)
'Socrates'                                                  TRUE (1.0, 1.0)
'Confucius'                                                FALSE (0.0, 0.0)



Since we know that $(\forall x) (native\_of\_greece(x) \lor resident\_of\_greece(x)) \Leftrightarrow grecian(x)$ is an axiom (so we know the truth value for each $x$) and truth values of first operand $(native\_of\_greece(x) \lor resident\_of\_greece(x))$ are infered by upward inference, we can then infer the truth value of $grecian(x)$ as we did in the beginning.